# Scraper for Companies
### This is the scraper for searching companies in given domain, the string of variable 'queries' is the key word for the search.
* Library Usage: selenium `pip install selenium`, beautifulsoup`pip install beautifulsoup4`
* Target website: angel.co
* Results are stored in companies.csv file under data folder
* Fields of output: Date of Join, Kind, Company webpage link on angel.co, location, market, name, investment raised, company size, stage, official website,the keyword used for searching this company
* Reference:https://github.com/rodrigosnader/angel-scraper

In [ ]:
#queries = ['computer vision', 'image understanding']
queries = ['computer vision']

In [1]:
import bs4 as bs
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def get_results(query, type_=None, stage=None, tech=None):
    
#     tech is possible only if type not None, same for others

    if type_:
        type_ = type_.replace(' ', '+')
        url = 'https://angel.co/companies' + '?company_types[]=' + type_
        if stage:
            url = url + '&stage=' + stage
            if tech:
                url = url + '&teches[]=' + tech
    else:  
        url = 'https://angel.co/companies' 

    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    
    driver = webdriver.Chrome(options=None)
    driver.set_window_position(3000,0)
    driver.get(url)
    time.sleep(5)
    
    search_box = driver.find_element_by_class_name("search-box")
    search_box.click()
    
    input_bar = driver.find_element_by_class_name('keyword-input')
    input_bar.send_keys(query)
    input_bar.send_keys(Keys.ENTER)
    time.sleep(3)

    while True:
        try:
            source = driver.page_source
            loadMoreButton = driver.find_element_by_class_name('more').click()
            time.sleep(4)
        except: 
            break
            
    driver.close()

    try:
        soup = bs.BeautifulSoup(source, 'lxml')
        result_list =  soup.find_all('div', {'class': 'results'})[0]
        results = result_list.find_all('div', {'data-_tn': 'companies/row'})
    except:
        print('Could not get results')
        return

    
    return results


def parse_results(results):
    df = pd.DataFrame(columns=['kind', 'link', 'name', 'pitch', 'joined', 'location', 'market', 'size', 'stage', 'raised'],
                  index=[0])
    for result in results[1:]:
        try:
            dic = {}
            try:
                dic['kind'] = result.a['data-type']
            except: dic['kind'] = None

            try:
                dic['link'] = result.a['href']
            except: dic['link'] = None

            try:
                dic['name'] = result.a['title']
            except: dic['name'] = None
                
            try:
                dic['website'] = result.find('div', {'data-column': 'website'}).text.split()[1]
            except: dic['website'] = None

            try:
                dic['pitch'] = result.find('div', {'class': 'pitch'}).text
            except: dic['pitch'] = None

            try:
                dic['joined'] = result.find('div', {'data-column': 'joined'}).text.split('Joined')[1].strip()
            except: dic['joined'] = None

            try:
                dic['location'] = result.find('div', {'data-column': 'location'}).text.split('Location')[1].strip()
            except: dic['location'] = None

            try:
                dic['market'] = result.find('div', {'data-column': 'market'}).text.split('Market')[1].strip()
            except: dic['market'] = None

            try:
                dic['size'] = result.find('div', {'data-column': 'company_size'}).text.split()[1]
            except: dic['size'] = None

            try:
                dic['stage'] = result.find('div', {'data-column': 'stage'}).text.split('Stage')[1].strip()
            except: dic['stage'] = None

            try:
                dic['raised'] = result.find('div', {'data-column': 'raised'}).text.split('Raised')[1].strip()
            except: dic['raised'] = None

            df = df.append(pd.DataFrame(dic, index=[0]))

        except: pass

    df = df.reset_index(drop=True)
    return df



def get_companies(query, type_=None, stage=None, tech=None):
    results = get_results(query, type_, stage, tech)
    if results:
        df = parse_results(results)
        return df

In [3]:
df = pd.DataFrame()
for query in queries:
    try:
        companies = get_companies(query)
        companies['query'] = query
        print('Done')
    except Exception as e: 
        companies = pd.DataFrame()
        print('Could not get companies')
        print('\n')
        print(e)
        print('\n')
        pass

    try:
        if df.empty:
            df = companies
        else:
            df = df.append(companies)
    except:
        print('Unable to append new data.')
        pass
                    
df = df.reset_index(drop=True)
df.to_csv('../00-data/companies.csv')
print(df.location.head())

C:\Users\FRE\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Done
0              NaN
1    New York City
2    Mountain View
3        Sunnyvale
4    San Francisco
Name: location, dtype: object


## Contributions 
-By own: 50%  
-By online resources: 50%  

## Citations
1. https://github.com/rodrigosnader/angel-scraper

## License

Copyright 2019 COPYRIGHT Yunan Shao

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.